In [1]:
import pandas as pd
import numpy as np

import sqlite3
db = "new_db.db"
con = sqlite3.connect(db)
cur = con.cursor()

from sklearn.linear_model import SGDClassifier

import os.path
from os import path

from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib

import pickle

import time
start = time.time()

In [2]:
def main():
    emails = list(cur.execute("""SELECT (
                                    COALESCE(email_from, '') || ' ' ||
                                    COALESCE(email_to, '') || ' ' ||
                                    COALESCE(email_cc, '') || ' ' ||
                                    COALESCE(email_bcc, '') || ' ' ||
                                    COALESCE(email_subject, '') || ' ' ||
                                    COALESCE(email_message, '')
                                )

                                FROM emails_main
                                WHERE folder_directory IS NOT NULL"""))

    emails = [item[0] for item in emails]

    emails_folder = list(cur.execute("""SELECT folder_directory 
                                        FROM emails_main
                                        WHERE folder_directory IS NOT NULL"""))

    emails_folder = [item[0] for item in emails_folder]

    emails_tobeprocessed = list(cur.execute("""SELECT (
                                                COALESCE(email_from, '') || ' ' ||
                                                COALESCE(email_to, '') || ' ' ||
                                                COALESCE(email_cc, '') || ' ' ||
                                                COALESCE(email_bcc, '') || ' ' ||
                                                COALESCE(email_subject, '') || ' ' ||
                                                COALESCE(email_message, '')
                                            )

                                            FROM emails_main
                                            WHERE folder_directory IS NULL"""))

    emails_tobeprocessed = [item[0] for item in emails_tobeprocessed]

    emails_tobeprocessed_messageid = list(cur.execute("""SELECT message_id
                                                         FROM emails_main
                                                         WHERE folder_directory IS NULL"""))

    emails_tobeprocessed_messageid = [item[0] for item in emails_tobeprocessed_messageid]

    emails_tobeprocessed_tuple = list(zip(emails_tobeprocessed_messageid, emails_tobeprocessed))

    X_train = emails
    y_train = emails_folder

    from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer

    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_train)
    
    X_train = vectorizer.transform(X_train)

    X_predict = [email[1] for email in emails_tobeprocessed_tuple]
    X_predict = vectorizer.transform(X_predict)

    # OneClassSVM - to weed out outliers
    from sklearn.svm import OneClassSVM
    oneclasssvm = OneClassSVM()
    oneclasssvm.fit(X_train)
    oneclass_preds = list(oneclasssvm.predict(X_predict))

    # Get indexes of outliers
    # Outliers = -1, Inliers = 1
    outliers_indexes = [i for i,x in enumerate(oneclass_preds) if x == -1]

    if len(oneclass_preds) != len(outliers_indexes):

        # Delete the outliers, in reverse order so it doesn't throw off the subsequent indexes
        for index in sorted(outliers_indexes, reverse=True):
            del emails_tobeprocessed_tuple[index]

        # New value for X_predict after deletion of outliers
        X_predict = [email[1] for email in emails_tobeprocessed_tuple]
        X_predict = vectorizer.transform(X_predict)

        model = SGDClassifier()
        model.fit(X_train, y_train)

        folder_directory = list(model.predict(X_predict))

        # Unzip tuple to lists
        message_id, email = zip(*emails_tobeprocessed_tuple)

        supervised_temp_df = pd.DataFrame({'message_id': message_id, 
                                           'folder_directory': folder_directory})

        # Create a temporary table to store the results of supervised learning
        supervised_temp_df.to_sql('supervised_temp', con, if_exists='replace')

        # Update folder_directory in emails table and delete temporary table for supervised learning
        cur.executescript("""UPDATE emails_main
                             SET folder_directory = (
                                SELECT folder_directory 
                                FROM supervised_temp 
                                WHERE message_id = emails_main.message_id)
                             WHERE emails_main.folder_directory IS NULL;

                             DROP TABLE supervised_temp;""")
        con.commit()    

In [3]:
main()

end = time.time()
print(end - start)

C:\Users\Jasmi\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Jasmi\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


24.2562358379364


In [4]:
con.close()